In [80]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [81]:
df_raw = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')

In [100]:
df_raw.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [101]:
df_raw.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

#### 1, 2, 3월 데이터만 추출

In [102]:
df = df_raw[(df_raw['REG_YYMM'] == 202001) | (df_raw['REG_YYMM'] == 202002) | (df_raw['REG_YYMM'] == 202003)]
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
20425415,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,3,345000,3
20425416,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,3,1903450,3
20425417,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,14,1520500,15
20425418,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,9,1239200,9
20425419,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,606700,4


In [103]:
df.fillna('세종시', inplace=True)

In [104]:
df.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [105]:
df.drop(['CARD_CCG_NM','HOM_SIDO_NM','HOM_CCG_NM'], axis=1, inplace=True)

In [106]:
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).sum().reset_index()

In [107]:
df.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202001,강원,건강보조식품 소매업,20s,1,1,6,168500,6
1,202001,강원,건강보조식품 소매업,20s,2,1,13,764646,13
2,202001,강원,건강보조식품 소매업,30s,1,2,72,12017410,73
3,202001,강원,건강보조식품 소매업,30s,2,2,88,9867625,89
4,202001,강원,건강보조식품 소매업,40s,1,3,90,10645775,91


# 군집 데이터 불러오기

In [108]:
cluster_info = pd.read_csv("./data/class_cluster.txt")

In [109]:
for i in cluster_info.iloc[4]:
    print(i)

['버스 운송업' '체인화 편의점']


# 5번 군집

In [110]:
class_list = ['버스 운송업', '체인화 편의점']

In [111]:
df = df[df['STD_CLSS_NM'].map(lambda x: x in class_list)]

In [112]:
df.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
232,202001,강원,버스 운송업,30s,1,2,3,30000,5
233,202001,강원,버스 운송업,50s,1,4,28,377200,40
234,202001,강원,버스 운송업,50s,2,4,7,92300,11
235,202001,강원,버스 운송업,60s,1,5,9,123700,15
236,202001,강원,버스 운송업,60s,2,5,3,25000,4


In [113]:
df.STD_CLSS_NM.unique()

array(['버스 운송업', '체인화 편의점'], dtype=object)

In [114]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [115]:
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

In [116]:
df.CARD_SIDO_NM.unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

# Label Encoding

In [117]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1720 entries, 232 to 38401
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CARD_SIDO_NM  1720 non-null   int32
 1   STD_CLSS_NM   1720 non-null   int32
 2   AGE           1720 non-null   int32
 3   SEX_CTGO_CD   1720 non-null   int64
 4   FLC           1720 non-null   int64
 5   CSTMR_CNT     1720 non-null   int64
 6   AMT           1720 non-null   int64
 7   CNT           1720 non-null   int64
 8   year          1720 non-null   int64
 9   month         1720 non-null   int64
dtypes: int32(3), int64(7)
memory usage: 127.7 KB


In [119]:
df.STD_CLSS_NM.unique()

array([0, 1])

# Preprocessing

In [120]:
# 변수명 지정
X_cols = list(df.columns)
X_cols.remove('AMT')

In [121]:
X = df.drop(['CSTMR_CNT','AMT','CNT'], axis=1)
y = np.log1p(df['AMT'])

In [122]:
# X 변수 Scale 적용
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [124]:
X_train

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
8419,10,1,2,1,1,2020,1
6669,8,0,4,2,4,2020,1
33299,9,1,2,1,3,2020,3
18814,7,0,4,2,4,2020,2
33914,10,1,1,1,2,2020,3
...,...,...,...,...,...,...,...
8417,10,1,1,2,1,2020,1
34702,11,1,3,1,3,2020,3
29734,4,1,6,2,5,2020,3
33922,10,1,2,2,3,2020,3


# Light Gradient Boosting Machine

In [125]:
import lightgbm as lgb

In [126]:
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)

In [127]:
params = {
            'learning_rate' : 0.1,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [128]:
model = lgb.train(params,
                  train_ds,
                  5000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.887995
[200]	valid_0's rmse: 0.770823
[300]	valid_0's rmse: 0.707613
[400]	valid_0's rmse: 0.659923
[500]	valid_0's rmse: 0.632433
[600]	valid_0's rmse: 0.612527
[700]	valid_0's rmse: 0.591583
[800]	valid_0's rmse: 0.571056
[900]	valid_0's rmse: 0.548389
[1000]	valid_0's rmse: 0.534715
[1100]	valid_0's rmse: 0.523083
[1200]	valid_0's rmse: 0.512306
[1300]	valid_0's rmse: 0.50219
[1400]	valid_0's rmse: 0.496559
[1500]	valid_0's rmse: 0.491607
[1600]	valid_0's rmse: 0.48646
[1700]	valid_0's rmse: 0.482105
[1800]	valid_0's rmse: 0.47981
[1900]	valid_0's rmse: 0.476472
[2000]	valid_0's rmse: 0.474139
[2100]	valid_0's rmse: 0.471738
[2200]	valid_0's rmse: 0.469715
[2300]	valid_0's rmse: 0.468875
[2400]	valid_0's rmse: 0.468182
[2500]	valid_0's rmse: 0.466805
[2600]	valid_0's rmse: 0.46602
Early stopping, best iteration is:
[2531]	valid_0's rmse: 0.465772


In [129]:
X.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year',
       'month'],
      dtype='object')

In [130]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
#CARD_CCG_NMs  = df['CARD_CCG_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = [2020]
months        = [4]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for AGE in AGEs:
            for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                    for year in years:
                        for month in months:
                            temp.append([CARD_SIDO_NM, STD_CLSS_NM,  AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=X.columns)

In [131]:
X.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year',
       'month'],
      dtype='object')

In [132]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,2,1,2,2020,4
1,0,0,2,1,4,2020,4
2,0,0,2,1,5,2020,4
3,0,0,2,1,1,2020,4
4,0,0,2,1,3,2020,4
...,...,...,...,...,...,...,...
2375,16,1,6,2,2,2020,4
2376,16,1,6,2,4,2020,4
2377,16,1,6,2,5,2020,4
2378,16,1,6,2,1,2020,4


In [133]:
# 예측
pred = model.predict(temp)

In [134]:
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [135]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,7.097769e+06
1,202004,0,1,1.020385e+10
2,202004,1,0,3.229665e+09
3,202004,1,1,1.307863e+11
4,202004,2,0,8.275890e+05
5,202004,2,1,2.945215e+10
6,202004,3,0,1.962873e+06
7,202004,3,1,1.621150e+10
8,202004,4,0,1.350726e+07
9,202004,4,1,1.156466e+10


In [136]:
encoders

{'CARD_SIDO_NM': LabelEncoder(),
 'STD_CLSS_NM': LabelEncoder(),
 'AGE': LabelEncoder()}

In [137]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [138]:
temp.to_csv('./data/clusters/cluster5.csv')